# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader
!pip install invisible-watermark>=0.2.0
!pip install --upgrade --no-cache-dir gdown

Tesla T4, 15360 MiB, 15101 MiB


https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth

## Install Requirements

In [3]:
!wget -q https://raw.githubusercontent.com/huggingface/diffusers/main/examples/dreambooth/train_dreambooth_lora_sdxl.py

In [4]:
!pip install -U -qq git+https://github.com/huggingface/diffusers.git
!pip install -qq accelerate tensorboard transformers ftfy gradio xformers
!pip install -qq "ipywidgets>=7,<8"
!pip install -qq bitsandbytes
!pip install -U --pre triton
!pip install wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached triton-2.0.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (63.3 MB)


## Imports

In [5]:
from natsort import natsorted
from glob import glob
from os import sep, makedirs
from os.path import exists, join, realpath

import gdown
import PIL
from PIL import Image

from os.path import exists, join, isfile, realpath, isdir, basename, dirname
from os import listdir, makedirs

from tqdm.auto import tqdm
from accelerate.utils import write_basic_config

from IPython.display import clear_output
from IPython.display import Image as ipImage
from IPython.display import HTML
from IPython import display as ipythondisplay
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline

import base64
import io
import imageio
from PIL import Image

import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from os.path import exists, join, isfile, realpath, isdir, basename, dirname
from os import listdir, makedirs, walk
import numpy as np

from tqdm.notebook import tqdm

import gradio as gr

from functools import partialmethod

In [6]:
DATA_DIR = '../datasets/generated_datasets'

if exists("/content/drive/MyDrive/ORT/Master/Codes/diffusion"):
  WORK_DIR = "/content/drive/MyDrive/ORT/Master/Codes/diffusion"
else:
  WORK_DIR = "/content/drive/MyDrive/ORT/Tesis/Codes/diffusion"

WORK_DIR

'/content/drive/MyDrive/ORT/Tesis/Codes/diffusion'

In [7]:
def get_dirs(dir_path: str):
    return [join(dir_path,d) for d in listdir(dir_path) if isdir(join(dir_path, d))]

def get_dir_files(dir_path: str):
    return [f for f in listdir(dir_path) if isfile(join(dir_path, f))]

In [8]:
# remove watermarking
class NoWatermark:
    def apply_watermark(self, img):
        return img

## HF token

In [9]:
!mkdir -p ~/.huggingface
HUGGINGFACE_TOKEN = ""
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

## Settings

In [10]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [11]:
# weights model folder
OUTPUT_DIR_Clusters = join(WORK_DIR, "retinopatia/models/stable-diffusion-xl_v1-lora-retinopatia_milds_clusters-finetuned")
OUTPUT_DIRS_Clusters = get_dirs(OUTPUT_DIR_Clusters)
OUTPUT_DIRS_Clusters

['/content/drive/MyDrive/ORT/Tesis/Codes/diffusion/retinopatia/models/stable-diffusion-xl_v1-lora-retinopatia_milds_clusters-finetuned/cluster_1_image',
 '/content/drive/MyDrive/ORT/Tesis/Codes/diffusion/retinopatia/models/stable-diffusion-xl_v1-lora-retinopatia_milds_clusters-finetuned/cluster_6_image',
 '/content/drive/MyDrive/ORT/Tesis/Codes/diffusion/retinopatia/models/stable-diffusion-xl_v1-lora-retinopatia_milds_clusters-finetuned/cluster_4_image',
 '/content/drive/MyDrive/ORT/Tesis/Codes/diffusion/retinopatia/models/stable-diffusion-xl_v1-lora-retinopatia_milds_clusters-finetuned/cluster_8_image',
 '/content/drive/MyDrive/ORT/Tesis/Codes/diffusion/retinopatia/models/stable-diffusion-xl_v1-lora-retinopatia_milds_clusters-finetuned/cluster_9_image',
 '/content/drive/MyDrive/ORT/Tesis/Codes/diffusion/retinopatia/models/stable-diffusion-xl_v1-lora-retinopatia_milds_clusters-finetuned/cluster_5_image',
 '/content/drive/MyDrive/ORT/Tesis/Codes/diffusion/retinopatia/models/stable-diffu

In [12]:
print(f"[*] Weights will be load from {OUTPUT_DIR_Clusters}")

[*] Weights will be load from /content/drive/MyDrive/ORT/Tesis/Codes/diffusion/retinopatia/models/stable-diffusion-xl_v1-lora-retinopatia_milds_clusters-finetuned


In [13]:
SEED = 52362

In [14]:
MODEL_NAME = "stabilityai/stable-diffusion-xl-base-1.0"

In [15]:
TOTAL_IMAGES_TO_GENERATE = 261

# Inference positive clusters model



In [16]:
for WEIGHTS_DIR in OUTPUT_DIRS_Clusters:
    # lora_model_id = join(OUTPUT_DIR, WEIGHTS_DIR)
    lora_model_id = WEIGHTS_DIR

    pipe = DiffusionPipeline.from_pretrained(MODEL_NAME, torch_dtype=torch.float16)
    pipe = pipe.to("cuda")
    pipe.load_lora_weights(lora_model_id)

    pipe.watermark = NoWatermark()

    print(f"### {WEIGHTS_DIR} ###")
    prompt = "an image of a glioma-brain-tumor MRI"
    pipe.set_progress_bar_config(disable=True)
    images = []
    ref_images = []

    IMAGE_FOLDER = realpath(join(WORK_DIR, DATA_DIR, 'retinopatia/diffusion/StableDiffusionV1/xl_v1-lora-milds_clusters_clip-finetuned'))
    if not exists(IMAGE_FOLDER):
        makedirs(IMAGE_FOLDER)
    cont = 0
    iterations = int(TOTAL_IMAGES_TO_GENERATE/len(OUTPUT_DIRS_Clusters))

    for i in tqdm(range(iterations), total=iterations):
        gi = pipe(prompt,
                    height=320,
                    width=320,
                    num_images_per_prompt=1,
                    num_inference_steps=24,
                    guidance_scale=7.5).images[0]

        gi = gi.resize((300, 300), Image.ANTIALIAS)
        imageio.imwrite(join(IMAGE_FOLDER, f"{basename(WEIGHTS_DIR)}{cont}.png"), gi)
        cont += 1


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

### /content/drive/MyDrive/ORT/Tesis/Codes/diffusion/retinopatia/models/stable-diffusion-xl_v1-lora-retinopatia_milds_clusters-finetuned/cluster_1_image ###


  0%|          | 0/26 [00:00<?, ?it/s]

<ipython-input-16-858b6d945fbb>:31: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  gi = gi.resize((300, 300), Image.ANTIALIAS)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

...

### /content/drive/MyDrive/ORT/Tesis/Codes/diffusion/retinopatia/models/stable-diffusion-xl_v1-lora-retinopatia_milds_clusters-finetuned/cluster_2_image ###


  0%|          | 0/26 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

### /content/drive/MyDrive/ORT/Tesis/Codes/diffusion/retinopatia/models/stable-diffusion-xl_v1-lora-retinopatia_milds_clusters-finetuned/cluster_7_image ###


  0%|          | 0/26 [00:00<?, ?it/s]